In [1]:
import json
import os
import csv
from collections import Counter
from estnltk import Text
from estnltk.converters import json_to_text
from pandas import DataFrame

In [2]:
stopid_fail = 'estonian-stopwords.txt'
lemmad_fail = 'estonian-stopwords-lemmas.txt'

stopid_s = []
with open(stopid_fail, 'r') as sf:
    for line in sf:
        stopid_s.append(line.strip())

stopid_l = []
with open(lemmad_fail, 'r') as lf:
    for line in lf:
        stopid_l.append(line.strip())

In [3]:
def speech_jsonist(dir):

    sekundid = 0
    koik_konet = []
    
    for fail in os.listdir(dir):
        f_nimi = os.path.join(dir, fail)
        if not fail.endswith('.json'):
            continue
        with open(f_nimi, 'r') as f:
       
            andmed = json.load(f)
     
            for section in andmed['sections']:
                if 'turns' in section.keys():
                    algus = float(section['start'])
                    lopp = float(section['end'])
                    aeg_kokku = lopp - algus
                    sekundid += aeg_kokku

                    for turn in section['turns']:
                        kone = turn['transcript']
                        koik_konet.append(kone)

    return sekundid/60, ' '.join(koik_konet)

In [4]:
def loendamine(kone):
    #es_ja_tei_isiku_asesonad = ['mina', 'meie', 'sina', 'teie']
    
    koik_sonad = []
    koik_lemmad = []

    partiklid = {}
    omadussonad = {}
    nimisonad = {}
    adverbid = {}
    verbid = {}
    asesonad = {}
    kaassonad = {}

    
    tekst = Text(kone).tag_layer('morph_analysis')
    for sone in tekst.morph_analysis:
        lemma = sone.lemma[0].lower()
        soneliik = sone.partofspeech[0]
                        
        if soneliik != 'Z':
            koik_lemmad.append(lemma)
            koik_sonad.append(sone.text.lower())
                

                        #if lemma not in stopid_l and koneliik != 'Z':
                            #if lemma == ',':
                                #print(koneliik)
                            #freq_counter[lemma] += 1

        vorm = sone.form[0]
        if soneliik == 'V':
                            #print(sone.text)
            verbid[lemma] = verbid.get(lemma, 0) + 1
                            
        elif soneliik == 'S':
            nimisonad[lemma] = nimisonad.get(lemma, 0) + 1
        elif soneliik == 'A':
            omadussonad[lemma] = omadussonad.get(lemma, 0) + 1
        elif soneliik == 'D':
            adverbid[lemma] = adverbid.get(lemma, 0) + 1
                            #print(sone.partofspeech)
        elif soneliik == 'P':
            asesonad[lemma] = asesonad.get(lemma, 0) + 1
            
        elif soneliik == 'I':
            partiklid[sone.text.lower()] = partiklid.get(sone.text.lower(), 0) + 1

        elif soneliik == 'K':
            kaassonad[lemma] = kaassonad.get(lemma, 0) + 1
            
    return koik_sonad, koik_lemmad, omadussonad, nimisonad, adverbid, verbid, asesonad, partiklid, kaassonad

In [5]:
def sagedus_loend(lemmad, stop_sonad):
    freq_counter = Counter()
    for lemma in lemmad:
        if lemma not in stop_sonad:
            freq_counter[lemma] += 1

    return freq_counter

In [6]:
minutid_mehed, kone_mehed = speech_jsonist('mehed')
print(minutid_mehed)

2590.5986666666695


In [7]:
minutid_naised, kone_naised = speech_jsonist('naised')
print(minutid_naised)

2010.3988333333323


In [8]:
print(minutid_mehed/60)

43.17664444444449


In [9]:
print(minutid_naised/60)

33.506647222222206


In [10]:
koik_sonad_mehed, koik_lemmad_mehed, omadussonad_mehed, nimisonad_mehed, adverbid_mehed, verbid_mehed, asesonad_mehed, partiklid_mehed, kaassonad_mehed = loendamine(kone_mehed)

In [11]:
koik_sonad_naised, koik_lemmad_naised, omadussonad_naised, nimisonad_naised, adverbid_naised, verbid_naised, asesonad_naised, partiklid_naised, kaassonad_naised = loendamine(kone_naised)

Sagedusloendid

In [12]:
def sageduse_fail(failinimi, loend, minutid):
    with open(failinimi, 'w', encoding='utf-8', newline='') as out_f:
        writer = csv.writer(out_f, delimiter=';')
    
  
        writer.writerow(['lemma', 'raw_count', 'freq_per_60_min'])

        for lemma, cnt in loend.most_common():
            freq_per_1000 = (cnt / minutid) * 60
            writer.writerow([lemma, cnt, f"{freq_per_1000:.3f}"])

In [13]:
filename_out = 'lemmade_sagedus_by_min_mehed_veel.csv'

sageduse_fail(filename_out, sagedus_loend(koik_lemmad_mehed, stopid_l), minutid_mehed)

In [14]:
filename_out = 'lemmade_sagedus_by_min_naised_veel.csv'

sageduse_fail(filename_out, sagedus_loend(koik_lemmad_naised, stopid_l), minutid_naised)

Bigrammid ja trigrammid:

In [15]:
def bi_ja_tri_grammid(lemmad, stop_lemmad, sonad, stop_sonad):

    bigram_counter_lemma = Counter()
    trigram_counter_lemma = Counter()
    
    for i in range(len(lemmad)-1):
        bigram = (lemmad[i].lower(), lemmad[i+1].lower())
        if bigram[0] != bigram[1]:
    
            if bigram[0] not in stop_lemmad and bigram[1] not in stop_lemmad:
                bigram_counter_lemma[bigram] += 1
            elif bigram[1] not in stop_lemmad:
                bigram_counter_lemma[bigram] += 1
    
    for i in range(len(lemmad) - 2):
        w1 = lemmad[i].lower()
        w2 = lemmad[i+1].lower()
        w3 = lemmad[i+2].lower()
        

        if w1 != w2 and w2 != w3 and w1 != w3:
            if w1 not in stop_lemmad and w2 not in stop_lemmad and w3 not in stop_lemmad:
                trigram = (w1, w2, w3)
                trigram_counter_lemma[trigram] += 1
            elif w1 not in stop_lemmad and  w3 not in stop_lemmad:
                trigram = (w1, w2, w3)
                trigram_counter_lemma[trigram] += 1
    
    print("Top 10 bigrams (lemmas):")
    for bg, cnt in bigram_counter_lemma.most_common(10):
        print(bg, cnt)
        
    print("\nTop 10 trigrams (lemmas):")
    for tg, cnt in trigram_counter_lemma.most_common(10):
        print(tg, cnt)
    

    bigram_counter_raw = Counter()
    trigram_counter_raw = Counter()
    
    for i in range(len(sonad)-1):
        bigram = (sonad[i].lower(), sonad[i+1].lower())
        if bigram[0] != bigram[1]:
            if bigram[0] not in stop_sonad and bigram[1] not in stop_sonad:
                bigram_counter_raw[bigram] += 1
            elif bigram[1] not in stop_sonad:
                bigram_counter_raw[bigram] += 1
    
    for i in range(len(sonad) - 2):
        w1 = sonad[i].lower()
        w2 = sonad[i+1].lower()
        w3 = sonad[i+2].lower()
        

        if w1 != w2 and w2 != w3 and w1 != w3:
            if w1 not in stop_sonad and w2 not in stop_sonad and w3 not in stop_sonad:
                trigram = (w1, w2, w3)
                trigram_counter_raw[trigram] += 1
            elif w1 not in stop_sonad and  w3 not in stop_sonad:
                trigram = (w1, w2, w3)
                trigram_counter_raw[trigram] += 1
    
    print("\nTop 10 bigrams (raw forms):")
    for bg, cnt in bigram_counter_raw.most_common(10):
        print(bg, cnt)
        
    print("\nTop 10 trigrams (raw forms):")
    for tg, cnt in trigram_counter_raw.most_common(10):
        print(tg, cnt)
    return bigram_counter_lemma, trigram_counter_lemma, bigram_counter_raw, trigram_counter_raw

In [16]:
bigrammid_lem_mehed, trigrammid_lem_mehed, bigrammid_raw_mehed, trigrammid_raw_mehed = bi_ja_tri_grammid(koik_lemmad_mehed, 
                                                                                                         stopid_l, koik_sonad_mehed, stopid_s)

Top 10 bigrams (lemmas):
('ei', 'teadma') 854
('mina', 'arvama') 656
('see', 'mõte') 531
('mina', 'ütlema') 346
('see', 'asi') 302
('saama', 'aru') 298
('mina', 'tahtma') 289
('mina', 'arust') 273
('kogu', 'aeg') 236
('mina', 'rääkima') 182

Top 10 trigrams (lemmas):
('eesti', 'puue', 'inimene') 28
('puue', 'inimene', 'koda') 27
('mees', 'ei', 'nutma') 20
('no', 'see', 'mõte') 17
('teadma', 'mis', 'asi') 13
('no', 'mina', 'ütlema') 12
('jutt', 'roheline', 'stuudio') 10
('ütlema', 'et', 'kuulma') 10
('tõde', 'ja', 'õigus') 9
('no', 'mina', 'arvama') 9

Top 10 bigrams (raw forms):
('ei', 'tea') 793
('ma', 'arvan') 597
('selles', 'mõttes') 467
('minu', 'arust') 272
('kogu', 'aeg') 234
('ma', 'ütlen') 145
('see', 'ongi') 127
('väga', 'hea') 122
('ma', 'tean') 122
('saan', 'aru') 118

Top 10 trigrams (raw forms):
('eesti', 'puuetega', 'inimeste') 22
('mehed', 'ei', 'nuta') 18
('no', 'selles', 'mõttes') 17
('jutud', 'rohelises', 'stuudios') 10
('no', 'ma', 'arvan') 9
('puuetega', 'inimeste',

In [17]:
bigrammid_lem_naised, trigrammid_lem_naised, bigrammid_raw_naised, trigrammid_raw_naised = bi_ja_tri_grammid(koik_lemmad_naised, 
                                                                                                             stopid_l, koik_sonad_naised, stopid_s)

Top 10 bigrams (lemmas):
('ei', 'teadma') 446
('mina', 'arvama') 374
('see', 'mõte') 300
('mina', 'tahtma') 291
('olema', 'tegelikult') 229
('mina', 'ütlema') 226
('mina', 'rääkima') 212
('see', 'inimene') 209
('olema', 'hästi') 201
('olema', 'vaja') 180

Top 10 trigrams (lemmas):
('tartu', 'noorsootöö', 'keskus') 19
('mees', 'ja', 'naine') 13
('noor', 'vaimne', 'tervis') 11
('ütlema', 'et', 'kuulma') 10
('hästi', 'palju', 'erinev') 9
('ema', 'ja', 'isa') 8
('teinekord', 'ikka', 'jälle') 8
('no', 'mina', 'arvama') 8
('no', 'mina', 'ütlema') 8
('puudutama', 'mina', 'saade') 7

Top 10 bigrams (raw forms):
('ei', 'tea') 415
('ma', 'arvan') 337
('selles', 'mõttes') 274
('et', 'tegelikult') 173
('kogu', 'aeg') 159
('on', 'tegelikult') 158
('on', 'hästi') 144
('ma', 'tahan') 136
('on', 'vaja') 128
('me', 'räägime') 117

Top 10 trigrams (raw forms):
('tartu', 'noorsootöö', 'keskuse') 10
('vaatasite', 'vaadake', 'teinekord') 9
('teinekord', 'ikka', 'jälle') 8
('no', 'ma', 'arvan') 8
('hästi', 

In [20]:
def bi_ja_tri_failid(filename_bigrams_lemma, filename_trigrams_lemma, filename_bigrams_raw, filename_trigrams_raw, 
                    bigram_counter_lemma, trigram_counter_lemma, bigram_counter_raw, trigram_counter_raw, total_minutes):
   
    with open(filename_bigrams_lemma, 'w', encoding='utf-8', newline='') as out_f:
        writer = csv.writer(out_f, delimiter=';')

        writer.writerow(['ngram', 'count', 'freq_per_60_min'])
        
        for bg, cnt in bigram_counter_lemma.most_common():

            freq_1000 = (cnt / total_minutes) * 60  
            

            bg_str = ' '.join(bg)
            

            writer.writerow([bg_str, cnt, f"{freq_1000:.4f}"])
    
    with open(filename_trigrams_lemma, 'w', encoding='utf-8', newline='') as out_f:
        writer = csv.writer(out_f, delimiter=';')
        # Заголовки CSV
        writer.writerow(['ngram', 'count', 'freq_per_60_min'])
        
        for tg, cnt in trigram_counter_lemma.most_common():

            freq_1000 = (cnt / total_minutes) * 60  

            tg_str = ' '.join(tg)
            

            writer.writerow([tg_str, cnt, f"{freq_1000:.4f}"])
    
    with open(filename_bigrams_raw, 'w', encoding='utf-8', newline='') as out_f:
        writer = csv.writer(out_f, delimiter=';')

        writer.writerow(['ngram', 'count', 'freq_per_60_min'])
        
        for bg, cnt in bigram_counter_raw.most_common():

            freq_1000 = (cnt / total_minutes) * 60  
            

            bg_str = ' '.join(bg)
            

            writer.writerow([bg_str, cnt, f"{freq_1000:.4f}"])
    
    with open(filename_trigrams_raw, 'w', encoding='utf-8', newline='') as out_f:
        writer = csv.writer(out_f, delimiter=';')
 
        writer.writerow(['ngram', 'count', 'freq_per_60_min'])
        
        for tg, cnt in trigram_counter_raw.most_common():
            freq_1000 = (cnt / total_minutes) * 60 
            

            tg_str = ' '.join(tg)
            
            writer.writerow([tg_str, cnt, f"{freq_1000:.4f}"])


In [21]:
filename_bigrams_lemma_mehed = 'n_grammid/bigrams_lemma_per_60_minutes_mehed.csv'
filename_trigrams_lemma_mehed = 'n_grammid/trigrams_lemma_per_60_minutes_mehed.csv'
filename_bigrams_raw_mehed = 'n_grammid/bigrams_raw_per_60_minutes_mehed.csv'
filename_trigrams_raw_mehed = 'n_grammid/trigrams_raw_per_60_minutes_mehed.csv'

filename_bigrams_lemma_naised = 'n_grammid/bigrams_lemma_per_60_minutes_naised.csv'
filename_trigrams_lemma_naised = 'n_grammid/trigrams_lemma_per_60_minutes_naised.csv'
filename_bigrams_raw_naised = 'n_grammid/bigrams_raw_per_60_minutes_naised.csv'
filename_trigrams_raw_naised = 'n_grammid/trigrams_raw_per_60_minutes_naised.csv'

In [22]:
bi_ja_tri_failid(filename_bigrams_lemma_mehed, filename_trigrams_lemma_mehed, filename_bigrams_raw_mehed, filename_trigrams_raw_mehed, 
                    bigrammid_lem_mehed, trigrammid_lem_mehed, bigrammid_raw_mehed, trigrammid_raw_mehed, minutid_mehed)

In [23]:
bi_ja_tri_failid(filename_bigrams_lemma_naised, filename_trigrams_lemma_naised, filename_bigrams_raw_naised, filename_trigrams_raw_naised, 
                    bigrammid_lem_naised, trigrammid_lem_naised, bigrammid_raw_naised, trigrammid_raw_naised, minutid_naised)

Sõnaliigid:

In [24]:
def sagedused(sonastik):
    kokku = 0
    for v in sonastik.values():
        kokku += v
    return kokku

In [25]:
def data_frame(minutid, sagedused_omadussonad, sagedused_nimisonad, sagedused_adverbid, sagedused_verbid, sagedused_asesonad,
              sagedused_partiklid, sagedused_kaassonad):
    df = DataFrame({ 
        "sõnaliigid": ["asesõnad",
        "omadussõnad",
        "nimisõnad",
        "adverbid",
        "verbid",
        "partiklid",
        "kaassõnad"],
        "sagedused": [
            sagedused_asesonad,
            sagedused_omadussonad,
            sagedused_nimisonad,
            sagedused_adverbid,
            sagedused_verbid,
            sagedused_partiklid,
            sagedused_kaassonad,
        ],
        "sagedused lemma": [
            ((sagedused_asesonad) / minutid) * 60,
            (sagedused_omadussonad / minutid) * 60,
            (sagedused_nimisonad / minutid) * 60,
            (sagedused_adverbid / minutid) * 60,
            (sagedused_verbid / minutid) * 60,
            (sagedused_partiklid / minutid) * 60,
            (sagedused_kaassonad / minutid) * 60,
        ]
    })
    
    df = df.sort_values(by='sagedused', ascending=False)
    return df


In [26]:
data_frame(minutid_mehed, sagedused(omadussonad_mehed), sagedused(nimisonad_mehed), sagedused(adverbid_mehed), sagedused(verbid_mehed), 
           sagedused(asesonad_mehed), sagedused(partiklid_mehed), sagedused(kaassonad_mehed))

,sõnaliigid,sagedused,sagedused lemma
4,verbid,76096,1762.434320
0,asesõnad,65096,1507.666954
3,adverbid,63573,1472.393254
2,nimisõnad,61114,1415.441167
1,omadussõnad,13730,317.995995
6,kaassõnad,5319,123.191602
5,partiklid,3000,69.482009


In [27]:
data_frame(minutid_naised, sagedused(omadussonad_naised), sagedused(nimisonad_naised), 
           sagedused(adverbid_naised), sagedused(verbid_naised), sagedused(asesonad_naised), sagedused(partiklid_naised), 
           sagedused(kaassonad_naised))

,sõnaliigid,sagedused,sagedused lemma
4,verbid,63136,1884.282828
3,adverbid,54396,1623.439064
0,asesõnad,54149,1616.067392
2,nimisõnad,50246,1499.583043
1,omadussõnad,12520,373.657201
6,kaassõnad,4364,130.242813
5,partiklid,1733,51.721081


In [28]:
def sageduse_fail(failinimi, freqs, minutid):
    with open(failinimi, 'w', encoding='utf-8', newline='') as out_f:
        writer = csv.writer(out_f, delimiter=';')

        writer.writerow(['lemma', 'raw_count', 'freq_per_60_min'])
        
        for lemma, count in sorted(freqs.items(), key=lambda item: item[1], reverse=True):
            freq_per_1000 = (count / minutid) * 60
            writer.writerow([lemma, count, f"{freq_per_1000:.3f}"])

In [29]:
sageduse_fail('sonaliigid/partiklid_mehed.csv', partiklid_mehed, minutid_mehed)
sageduse_fail('sonaliigid/verbid_mehed.csv', verbid_mehed, minutid_mehed)
sageduse_fail('sonaliigid/omadussonad_mehed.csv', omadussonad_mehed, minutid_mehed)
sageduse_fail('sonaliigid/nimisonad_mehed.csv', nimisonad_mehed, minutid_mehed)
sageduse_fail('sonaliigid/adverbid_mehed.csv', adverbid_mehed, minutid_mehed)

In [30]:
sageduse_fail('sonaliigid/partiklid_naised.csv', partiklid_naised, minutid_naised)
sageduse_fail('sonaliigid/verbid_naised.csv', verbid_naised, minutid_naised)
sageduse_fail('sonaliigid/omadussonad_naised.csv', omadussonad_naised, minutid_naised)
sageduse_fail('sonaliigid/nimisonad_naised.csv', nimisonad_naised, minutid_naised)
sageduse_fail('sonaliigid/adverbid_naised.csv', adverbid_naised, minutid_naised)

Esimese ja teiste isiku asesonad:

In [31]:
es_ja_tei_isiku_asesonad = ['mina', 'meie', 'sina', 'teie']

kokku_asesonad_mehed = 0
kokku_es_ja_tei_isiku_asesonad_mehed = 0

for k, v in asesonad_mehed.items():
    kokku_asesonad_mehed += v
    if k in es_ja_tei_isiku_asesonad:
        kokku_es_ja_tei_isiku_asesonad_mehed += v

print((kokku_es_ja_tei_isiku_asesonad_mehed/kokku_asesonad_mehed) * 100)
print((kokku_es_ja_tei_isiku_asesonad_mehed/minutid_mehed) * 60)

30.175433206341403
454.9450345840262


In [32]:
kokku_asesonad_naised = 0
kokku_es_ja_tei_isiku_asesonad_naised = 0

for k, v in asesonad_naised.items():
    kokku_asesonad_naised += v
    if k in es_ja_tei_isiku_asesonad:
        kokku_es_ja_tei_isiku_asesonad_naised += v

print((kokku_es_ja_tei_isiku_asesonad_naised/kokku_asesonad_naised) * 100)
print((kokku_es_ja_tei_isiku_asesonad_naised/minutid_naised) * 60)

30.80019944966666
497.75198005901507


Siin püüan välja mõelda, kuidas kõige paremini eituste ja verbivormidega töötada. Lõpetan selle pühapäevaks.

In [33]:
tekst = Text(kone_mehed).tag_layer('morph_analysis')
for i, sone in enumerate(tekst.morph_analysis):
    lemma = sone.lemma[0].lower()
    soneliik = sone.partofspeech[0]
        
    if soneliik == 'V' and sone.form[0] == 'nud':
        print(sone.text, sone.form)

asunud ['nud', '', 'pl n', 'sg n']
elanud ['nud', '', 'pl n', 'sg n']
jõudnud ['nud', '', 'sg n', 'pl n']
õppinud ['nud', '', 'pl n', 'sg n']
teinud ['nud', '', 'pl n', 'sg n']
teinud ['nud', '', 'pl n', 'sg n']
palunud ['nud', '', 'sg n', 'pl n']
juhtunud ['nud', '', 'pl n', 'sg n']
saanud ['nud', '', 'pl n', 'sg n']
saanud ['nud', '', 'pl n', 'sg n']
käinud ['nud', '', 'pl n', 'sg n']
tahtnud ['nud', 'pl n', 'sg n', '']
mõjutanud ['nud', 'sg n', '', 'pl n']
osanud ['nud', 'pl n', 'sg n', '']
osanud ['nud', 'pl n', 'sg n', '']
näinud ['nud', '', 'sg n', 'nud', 'pl n']
muutunud ['nud', '', 'sg n', 'pl n']
muutunud ['nud', '', 'sg n', 'pl n']
näinud ['nud', '', 'sg n', 'nud', 'pl n']
muutunud ['nud', '', 'sg n', 'pl n']
muutunud ['nud', '', 'sg n', 'pl n']
kadunud ['nud', '', 'pl n', 'sg n']
tundnud ['nud', '', 'sg n', 'pl n']
muutunud ['nud', '', 'sg n', 'pl n']
muutunud ['nud', '', 'sg n', 'pl n']
puutunud ['nud', '', 'sg n', 'pl n']
teinud ['nud', '', 'pl n', 'sg n']
olnud ['nud', ''

In [34]:
tekst = Text(kone_mehed).tag_layer('morph_analysis')
for sentence in tekst.sentences:
    sonad = sentence.words
    for i, sone in enumerate(sonad):
        if sone.partofspeech[0] == 'V' and sone.form[0] == 'nud':
            print(sonad[i-1].text, sone.text)

oled asunud
, elanud
ringiga jõudnud
ilmselt õppinud
intervjuusid teinud
indekat teinud
olen palunud
ei juhtunud
ei saanud
ei saanud
ei käinud
ka tahtnud
negatiivselt mõjutanud
ei osanud
ei osanud
elu näinud
ka muutunud
see muutunud
olen näinud
palju muutunud
kõvasti muutunud
kuskile kadunud
ei tundnud
ei muutunud
oleks muutunud
ei puutunud
ei teinud
kunagi olnud
poleks olnud
ei olnud
ei läinud
ei hoolinud
ei olnud
ka käinud
ei olnud
Ei olnud
on jäänud
ei laulnud
ei olnud
ei teadnud
ei julgenud
ei laulnud
ei olnud
ei osanud
ei olnud
ei olnud
ei saanud
ei mõelnud
ei saanud
ei läinud
ei saanud
ei saanud
ka mõelnud
natukene kahetsenud
oleks pidanud
maapoiss kasvanud
oleks teinud
on tekkinud
elust võõrdunud
ei õppinud
ei läinud
ei läinud
ei olnud
oleks võinud
oleks võinud
sinna pannud
ei meeldinud
ei olnud
ei häirinud
ei olnud
ei olnud
on olnud
ei võitnud
absoluutselt seganud
ei pakkunud
ei pakkunud
me olnud
ei kaotanud
Ei kujutanud
ei teadnud
ei olnud
tööd teinud
ei olnud
tööd teinud
ei o

In [35]:
print(type(koik_lemmad_mehed))

<class 'list'>


In [36]:
eitused = ['ei', 'ära', 'ärge', 'mitte']

for inx,lemma in enumerate(koik_lemmad_mehed):
    
    #if lemma.startswith('eba') or lemma.startswith('mitte'):
        #print(lemma)

    #if lemma.endswith('tu'):
    #if lemma.endswith('mata') or lemma.endswith('matu'):
    if lemma in eitused:
        print(lemma, koik_lemmad_mehed[inx-1])

ei üldse
ära küsimus
ei väga
ei mina
ära kolima
ära loom
ei nagu
ei mina
ei et
ei vanainimene
ei tema
mitte mis
ei mina
ei mina
ei mina
ei mina
ei õde
ei mina
ei mina
ei mina
ära kolima
ära surema
ei inimene
ära surema
ei enam
ära raha
ära raha
ära maa
ei see
ei olema
ära panema
ei hariduselu
ei mina
ei kool
ära remont
ei mina
ei mina
mitte küll
ei mitte
ei mina
ei eiütleja
ei mina
ei õnn
ei mina
ei pahandus
ei mina
ei poiss
ei kus
ei teistsugune
mitte sest
ei kõik
ei mina
ei siis
ei mina
ei veel
ei väga
ei kes
ei mina
ei see
ei mina
mitte võima
mitte miks
ei roll
ei mina
ei mäletama
ei ka
ei mina
ei et
ei roll
ära üldpilt
ei mina
ei peaosa
ära miski
ära miski
mitte tarkus
ei see
ei sõber
ei tegelikult
ei sõber
ei saama
mitte praegu
ei mina
ei et
ei mina
ei mina
ei tegelikult
ei mina
ei mina
ei mina
ära unustama
ära unustama
ei kõik
ei tunne
ei mina
ei mina
ei maa
ei ja
ei mina
ei mina
ei mets
ei eesti
mitte tegelikult
ei sina
ei ei
ei siin
ei veel
ei mina
ei sina
ei mina
ära kaitseväg